In [3]:
import numpy as np


In [5]:
import tensorflow as tf

In [6]:
rng=np.random.default_rng(2)
X=rng.random(400).reshape(-1,2)
X[:,1]=X[:,1]*4+11.5
X[:,0]=X[:,0]*(285-150)+150
Y=np.zeros(len(X))

i=0
for t,d in X:
    y=-3/(260-175)*t+21
    if(t>175 and t<260 and d>12 and d<15 and d<=y):
        Y[i]=1
    else:
        Y[i]=0
    i+=1

In [7]:
Y=Y.reshape(-1,1)

In [8]:
print(f"Temperature Max, Min pre normalization: {np.max(X[:,0]):0.2f}, {np.min(X[:,0]):0.2f}")
print(f"Duration    Max, Min pre normalization: {np.max(X[:,1]):0.2f}, {np.min(X[:,1]):0.2f}")

Temperature Max, Min pre normalization: 284.99, 151.32
Duration    Max, Min pre normalization: 15.45, 11.51


In [9]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [10]:
norm_l=tf.keras.layers.Normalization(axis=-1)

In [12]:
norm_l.adapt(X)
Xn=norm_l(X)

In [13]:
print(f"Temperature Max Min post normalization : {np.max(Xn[:,0]):0.2f},{np.min(Xn[:,0]):0.2f}")
print(f"Temperature Max Min post normalization : {np.max(Xn[:,1]):0.2f},{np.min(Xn[:,1]):0.2f}")

Temperature Max Min post normalization : 1.66,-1.69
Temperature Max Min post normalization : 1.79,-1.70


In [14]:
Xt=np.tile(Xn,(1000,1))
Yt=np.tile(Y,(1000,1))

In [15]:
print(Xt.shape)
print(Yt.shape)

(200000, 2)
(200000, 1)


In [16]:
tf.random.set_seed(1234)
Model=Sequential([
    tf.keras.Input(shape=(2,)),
    Dense(3,activation='sigmoid',name='layer1'),
    Dense(1,activation='sigmoid',name='layer2')
])

In [17]:
Model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 layer1 (Dense)              (None, 3)                 9         
                                                                 
 layer2 (Dense)              (None, 1)                 4         
                                                                 
Total params: 13
Trainable params: 13
Non-trainable params: 0
_________________________________________________________________


In [18]:
W1, b1 = Model.get_layer("layer1").get_weights()
W2, b2 = Model.get_layer("layer2").get_weights()
print(f"W1{W1.shape}:\n", W1, f"\nb1{b1.shape}:", b1)
print(f"W2{W2.shape}:\n", W2, f"\nb2{b2.shape}:", b2)

W1(2, 3):
 [[ 0.38634133 -0.51666945 -0.6838556 ]
 [ 0.12615621 -0.3528415  -0.90096056]] 
b1(3,): [0. 0. 0.]
W2(3, 1):
 [[-0.13020802]
 [ 0.69021726]
 [-0.38586217]] 
b2(1,): [0.]


In [19]:
Model.compile(
    loss=tf.keras.losses.BinaryCrossentropy(),
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.01),
)

In [20]:
Model.fit(
Xt,Yt,
epochs=10
)

Epoch 1/10
6250/6250 [==============================] - 10s 2ms/step - loss: 0.1857
Epoch 2/10
6250/6250 [==============================] - 9s 1ms/step - loss: 0.1284
Epoch 3/10
6250/6250 [==============================] - 9s 1ms/step - loss: 0.1162
Epoch 4/10
6250/6250 [==============================] - 9s 1ms/step - loss: 0.0504
Epoch 5/10
6250/6250 [==============================] - 9s 1ms/step - loss: 0.0174
Epoch 6/10
6250/6250 [==============================] - 9s 1ms/step - loss: 0.0113
Epoch 7/10
6250/6250 [==============================] - 9s 1ms/step - loss: 0.0079
Epoch 8/10
6250/6250 [==============================] - 9s 1ms/step - loss: 0.0056
Epoch 9/10
6250/6250 [==============================] - 9s 1ms/step - loss: 0.0041
Epoch 10/10
6250/6250 [==============================] - 9s 1ms/step - loss: 0.0030


In [22]:
X_test=np.array([
    [200,13.9],  
    [200,17]])
X_testn=norm_l(X_test)
predictions=Model.predict(X_testn)
print(f"the predictions are ->{predictions}")

1/1 [==============================] - 0s 26ms/step
the predictions are ->[[9.76177514e-01]
 [1.18231415e-04]]


In [24]:
y_hat=np.zeros_like(predictions)
for i in range(len(predictions)):
    if predictions[i]>=0.5:
        y_hat[i]=1;
    else:
        y_hat[i]=0;
print(f"decisions=\n{y_hat}")

decisions=
[[1.]
 [0.]]


In [ ]:
done